## Import packages

In [ ]:
import ck.kernel as ck
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import json
import os

In [ ]:
print "Collective Knowledge: v%s" % ck.version({})['version_str']
print "pandas: v%s" % pd.version.version
print "NumPy: v%s" % np.version.version
print "Matplotlib: v%s" % matplotlib.__version__
print "JSON: v%s" % json.__version__

## Find results

In [ ]:
dataset = 'SGEMM_NT'
data_uoa =  dataset + '-explore-f-n'
module_uoa = 'experiment'

In [ ]:
r=ck.access({'action':'list_points', 'module_uoa':module_uoa, 'data_uoa':data_uoa})
if r['return']>0:
  print ("Error: %s" % r['error'])
  exit(1)

## Show results

### Table

In [ ]:
data_list  = []
index_list = []

for point in r['points']:
    with open(os.path.join(r['path'], 'ckp-%s.flat.json' % point)) as point_file:
        point_data = json.load(point_file)
    # Data.    
    Gflops_per_s = point_data.get("##characteristics#run#run_time_state#EXECUTION#Gflops/s#all")
    data_list.append(Gflops_per_s)
    # Index.
    cl_file = point_data.get("##characteristics#run#run_time_state#METADATA#file#all_unique")[0]
    lws_j = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_j#all_unique")[0]
    lws_i = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_i#all_unique")[0]
    local_work_size = ('(%s, %s)' % (lws_j, lws_i))
    matrix_order = point_data.get("##characteristics#run#run_time_state#CMD_LINE_ARGS#matrix_order#all_unique")[0]
    index_list.append((cl_file, local_work_size, matrix_order))

In [ ]:
mi = pd.MultiIndex.from_tuples(names=['OpenCL file', 'Local work size', 'Matrix order'], tuples=index_list)
df = pd.DataFrame(data=data_list, index=mi).sortlevel(level='OpenCL file')

In [ ]:
df

### Plot

In [ ]:
%matplotlib inline

In [ ]:
mean = df.mean(axis=1)
std = df.std(axis=1)
ymax = np.int64(df.max().max() + 1.0)

In [ ]:
plot = mean \
    .unstack(level='Local work size').unstack(level='OpenCL file') \
    .plot(yerr=std.unstack(level='Local work size').unstack(level='OpenCL file'),
        title='Gflops/s vs Matrix order',
        kind='bar', figsize=(12,8), colormap=matplotlib.cm.autumn,
        ylim=(0, ymax), yticks=range(ymax))

## Dump results for paper

### Table

In [ ]:
# A bug in pandas 0.16.1 prevents us from using MultiIndex when dumping to LaTeX.
mi_tex = pd.MultiIndex.from_tuples(tuples=index_list)
df_tex = pd.DataFrame(data=data_list, index=mi_tex).sortlevel(level=0)
# Add mean and std columns.
df_tex['mean'] = mean
df_tex['std'] = std

with open('%s_tmp.tex' % data_uoa, 'w') as tex_file:
    tex_file.write(df_tex.to_latex())

### Plot

In [ ]:
plot.get_figure().savefig('%s_tmp.pdf' % data_uoa)
# plot.get_figure().savefig('%s_tmp.png' % data_uoa)